# Problem Definition

In [ ]:
class Problem(object):
  """Problem definition"""

  def __init__(self, initial=None, goal=None):
    """Create a Problem instance, given an initial state and a goal state"""
    self.initial = initial
    self.goal = goal

  def actions(self, state): raise NotImplementedError
  def result(self, state, action): raise NotImplementedError
  def is_goal(self, state): return state == self.goal
  def action_cost(self, s, a, s1): return 1
  def h(self, node): return 0

In [ ]:
class Node():
  """A Node in a search tree"""

  def __init__(self, state, parent=None, action=None, path_cost=0):
    """Create a search tree Node, derived from a parent by an action"""
    self.state = state
    self.parent = parent
    self.action = action
    self.path_cost = path_cost
    self.depth = 0
    if parent:
      self.depth = parent.depth + 1

  def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))

  def expand(self, problem):
    """List the nodes reachable in one step from this node"""
    return  [self.child_node(problem, action) for action in problem.actions(self.state)]

  def child_node(self, problem, action):
    """Given an action, visualize the next node of the search tree"""
    next_state = problem.result(self.state, action)
    next_node = next_node = Node(next_state, self, action, self.path_cost + problem.action_cost(self.state, action, next_state))
    return next_node

# Frontier Data Structures


In [ ]:
from collections import deque
FIFOQueue = deque # First-in First-out Queue
LIFOQueue = list # Last-in First-out Queue

import heapq # Heap data structure
class PriorityQueue():
  """A queue in which the item with minimum f(item) is always popped first"""

  def __init__(self, items=(), key=lambda x: x):
    """Create the priority queue given a score function key, and a set of items"""
    self.key = key # function for finding the score given the item
    self.items = [] # a heap of (score, item) pairs
    for item in items:
      self.add(item)

  def add(self, item):
    """Add item to the queue"""
    pair = (self.key(item), item)
    heapq.heappush(self.items, pair)

  def pop(self):
    """Pop and return the item with min f(item) value"""
    return heapq.heappop(self.items)[1]

# Breath-first Search Algorithm

In [ ]:
def breadth_first_search(problem):
  """Search shallowest nodes in the search tree first"""
  node = Node(problem.initial)
  if problem.is_goal(problem.initial):
    return node
  frontier = FIFOQueue([node])
  reached = {problem.initial}
  while frontier:
    node = frontier.pop()
    for child in node.expand(problem):
      s = child.state
      if problem.is_goal(s):
        return child
      if s not in reached:
        reached.add(s)
        frontier.appendleft(child)

  import math
  return Node('failure', path_cost=math.inf)

# Depth-first Search Algorithm

In [ ]:
def is_cycle(node, k=30):
  "Does this node form a cycle of length k or less?"
  def find_cycle(ancestor, k):
    return ancestor is not None and k > 0 and (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1))
  return find_cycle(node.parent, k)

In [ ]:
def depth_first_search_recursive(problem, node=None):
  if node is None:
    node = Node(problem.initial)
  if problem.is_goal(node.state):
    return node
  elif is_cycle(node):
    import math
    return Node('failure', path_cost=math.inf)
  else:
    for child in node.expand(problem):
      result = depth_first_search_recursive(problem, child)
      if result:
        return result
    import math
    return Node('failure', path_cost=math.inf)

In [ ]:
def depth_first_search_iterative(problem):
  "Search deepest nodes in the search tree first"
  frontier = LIFOQueue([Node(problem.initial)])
  import math
  result = Node('failure', path_cost=math.inf)
  while frontier:
    node = frontier.pop()
    if problem.is_goal(node.state):
      return node
    elif not is_cycle(node):
      for child in node.expand(problem):
        frontier.append(child)
  return result

# Depth-limited Search Algorithm

In [ ]:
def depth_limited_search(problem, limit=10):
  "Search deepest nodes in the search tree first"
  frontier = LIFOQueue([Node(problem.initial)])
  import math
  result = Node('failure', path_cost=math.inf)
  while frontier:
    node = frontier.pop()
    if problem.is_goal(node.state):
      return Node('cutoff', path_cost=math.inf)
    elif len(node) >= limit:
      result = Node('cutoff', path_cost=math.inf)
    elif not is_cycle(node):
      for child in node.expand(problem):
        frontier.append(child)
  return result

In [ ]:
def iterative_deepening_search(problem):
  "Do depth-limited search with increasing depth limits"
  import sys, math
  for limit in range(1, sys.maxsize):
    result = depth_limited_search(problem, limit)
    if result != Node('cutoff', path_cost=math.inf):
      return result

# Best-first Search Algorithms

Best-first search is a class of search algorithms, which explores a graph by expanding the most promising node chosen according to a specified rule.

In [ ]:
def best_first_search(problem, f):
  "Search nodes with minimum f(node) value first"
  node = Node(problem.initial)
  frontier = PriorityQueue([node], key=f)
  reached = {problem.initial: node}
  while frontier:
    node = frontier.pop()
    if problem.is_goal(node.state):
      return node
    for child in node.expand(problem):
      s = child.state
      if s not in reached or child.path_cost < reached[s].path_cost:
        reached[s] = child
        frontier.add(child)

  import math
  return Node('failure', path_cost=math.inf)

In [ ]:
def breadth_first_search_bfs(problem):
  "Search shallowest nodes in the search tree first; using best-first"
  return best_first_search(problem, f=len)

In [ ]:
def g(n): return n.path_cost
def uniform_cost_search(problem):
  "Search nodes with minimum path cost first."
  return best_first_search(problem, f=g)

In [ ]:
def depth_first_search_bfs(problem):
  "Search deepest nodes in the search tree first; using best-first."
  return best_first_search(problem, f=lambda n: -len(n))

In [ ]:
def greedy_bfs(problem, h=None):
  """Search nodes with minimum h(n)."""
  h = h or problem.h
  return best_first_search(problem, f=h)

In [ ]:
def g(n): return n.path_cost
def astar_search(problem, h=None):
  """Search nodes with minimum f(n) = g(n) + h(n)."""
  h = h or problem.h
  return best_first_search(problem, f=lambda n: g(n) + h(n))

In [ ]:
def g(n): return n.path_cost
def weighted_astar_search(problem, h=None, weight=1.4):
  """Search nodes with minimum f(n) = g(n) + weight * h(n)."""
  h = h or problem.h
  return best_first_search(problem, f=lambda n: g(n) + weight * h(n))

# Exercies

# 8-Puzzle Problem

In [ ]:
class EightPuzzle(Problem):

  def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)):
    self.initial = initial
    self.goal = goal

  def actions(self, state):
    # the state is considered as a tuple of length 9 --> 9 blicks with a blank one
    moves = ((1, 3), (0, 2, 4), (1, 5), (0, 4, 6), (3, 1, 5, 7), (2, 4, 8), (3, 7), (4, 6, 8), (5, 7)) # there can I move the value from the index-position
    blank = state.index(0) # get the index of the value 0 into the tuple
    return moves[blank] # return the available moves in that index-positionù

  def result(self, state, action):
    s = list(state)
    blank = state.index(0)
    s[action], s[blank] = s[blank], s[action]
    return tuple(s)

In [ ]:
def path_states(node):
  "The sequence of states to get to this node."
  import math
  if node in (Node('cutoff', path_cost=math.inf), Node('failure', path_cost=math.inf), None): return []
  return path_states(node.parent) + [node.state]

In [ ]:
e1 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))

In [ ]:
for s in path_states(breadth_first_search(e1)): print(s)

(8, 6, 7, 2, 5, 4, 3, 0, 1)
(8, 6, 7, 2, 0, 4, 3, 5, 1)
(8, 6, 7, 2, 4, 0, 3, 5, 1)
(8, 6, 7, 2, 4, 1, 3, 5, 0)
(8, 6, 7, 2, 4, 1, 3, 0, 5)
(8, 6, 7, 2, 0, 1, 3, 4, 5)
(8, 0, 7, 2, 6, 1, 3, 4, 5)
(0, 8, 7, 2, 6, 1, 3, 4, 5)
(2, 8, 7, 0, 6, 1, 3, 4, 5)
(2, 8, 7, 3, 6, 1, 0, 4, 5)
(2, 8, 7, 3, 6, 1, 4, 0, 5)
(2, 8, 7, 3, 0, 1, 4, 6, 5)
(2, 0, 7, 3, 8, 1, 4, 6, 5)
(0, 2, 7, 3, 8, 1, 4, 6, 5)
(3, 2, 7, 0, 8, 1, 4, 6, 5)
(3, 2, 7, 4, 8, 1, 0, 6, 5)
(3, 2, 7, 4, 8, 1, 6, 0, 5)
(3, 2, 7, 4, 0, 1, 6, 8, 5)
(3, 2, 7, 4, 1, 0, 6, 8, 5)
(3, 2, 0, 4, 1, 7, 6, 8, 5)
(3, 0, 2, 4, 1, 7, 6, 8, 5)
(3, 1, 2, 4, 0, 7, 6, 8, 5)
(3, 1, 2, 4, 7, 0, 6, 8, 5)
(3, 1, 2, 4, 7, 5, 6, 8, 0)
(3, 1, 2, 4, 7, 5, 6, 0, 8)
(3, 1, 2, 4, 0, 5, 6, 7, 8)
(3, 1, 2, 0, 4, 5, 6, 7, 8)
(0, 1, 2, 3, 4, 5, 6, 7, 8)


# 8-Queens Puzzle Problem